# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

c:\Users\Igor\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
C:\Users\Igor\AppData\Local\Temp\ipykernel_26496\676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be r

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low'])
).assign(
    rolling_avg_dd = lambda x: x['returns'].rolling(10).mean()
))

C:\Users\Igor\AppData\Local\Temp\ipykernel_26496\2675210900.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [5]:
feat_df = dd_feat.compute()
feat_df['rolling_avg'] = feat_df['returns'].rolling(10).mean()
feat_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,returns,hi_lo_range,rolling_avg_dd,rolling_avg
ticker,,,,,,,,,,,,,,
A,2013-12-02 00:00:00-05:00,34.930990,35.061644,34.695808,34.761135,2039962,0.0,0.0,2013,NaN,NaN,0.365836,NaN,NaN
A,2013-12-03 00:00:00-05:00,34.571688,34.735011,34.336506,34.578220,3462706,0.0,0.0,2013,34.761135,-0.005262,0.398505,NaN,NaN
A,2013-12-04 00:00:00-05:00,34.519429,35.166180,34.441037,35.002857,3377288,0.0,0.0,2013,34.578220,0.012280,0.725143,NaN,NaN
A,2013-12-05 00:00:00-05:00,34.852600,35.146576,34.702345,34.950592,2530939,0.0,0.0,2013,35.002857,-0.001493,0.444231,NaN,NaN
A,2013-12-06 00:00:00-05:00,35.120459,35.884796,35.120459,35.819469,4268513,0.0,0.0,2013,34.950592,0.024860,0.764337,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,127.913548,128.386458,126.642582,128.297791,1871700,0.0,0.0,2024,127.529305,0.006026,1.743877,0.000828,0.000828
YUM,2024-01-26 00:00:00-05:00,128.002213,128.760851,126.770665,127.184464,1157000,0.0,0.0,2024,128.297791,-0.008678,1.990186,0.000115,0.000115
YUM,2024-01-29 00:00:00-05:00,127.194317,128.622910,126.396261,128.622910,1482100,0.0,0.0,2024,127.184464,0.011310,2.226649,0.001145,0.001145


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it wasn't necessary to to convert to pandas to calculate the moving average return. It would have been better to do it in Dask, because it would speed up the calculation due to paralellization capabilities of Dask library. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.